In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures

In [2]:
class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

In [3]:
wine = pd.read_csv('wine.csv', sep=';')
wine.info()
wine.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1199 non-null   float64
 1   volatile acidity      1199 non-null   float64
 2   citric acid           1199 non-null   float64
 3   residual sugar        1199 non-null   float64
 4   chlorides             1199 non-null   float64
 5   free sulfur dioxide   1199 non-null   float64
 6   total sulfur dioxide  1199 non-null   int64  
 7   density               1199 non-null   float64
 8   pH                    1199 non-null   float64
 9   sulphates             1199 non-null   float64
 10  alcohol               1199 non-null   float64
 11  quality               1199 non-null   int64  
dtypes: float64(10), int64(2)
memory usage: 112.5 KB


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34,0.9978,3.51,0.56,9.4,5


In [4]:
wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000
mean,8.625271,0.519133,0.293353,2.564470,0.089266,15.242702,46.884070,0.997059,3.298582,0.665738,10.383069,5.664721
std,1.781795,0.179208,0.196751,1.264441,0.048310,10.210406,33.949177,0.001878,0.156161,0.175921,1.091891,0.809593
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.300000,0.390000,0.120000,1.900000,0.071000,7.000000,21.000000,0.996000,3.195000,0.560000,9.500000,5.000000
50%,8.300000,0.500000,0.290000,2.200000,0.080000,13.000000,38.000000,0.997020,3.300000,0.620000,10.000000,6.000000
75%,9.600000,0.630000,0.450000,2.700000,0.092000,21.000000,63.000000,0.998175,3.390000,0.735000,11.000000,6.000000
max,15.900000,1.330000,1.000000,15.500000,0.611000,68.000000,289.000000,1.003200,3.900000,2.000000,14.900000,8.000000


In [5]:
X = wine.drop('quality', axis= 1)
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34,0.9978,3.51,0.56,9.4


In [6]:
y = wine.quality
y.head()

0    5
1    5
2    5
3    6
4    5
Name: quality, dtype: int64

    a)

In [7]:
model_full = sm.OLS(y,X).fit()
mf_score = np.mean(cross_val_score(SMWrapper(sm.OLS), X, y, scoring='r2', cv=5))
print(f'Full Attributes CV Score: {mf_score}')
pd.DataFrame(model_full.params, columns=['coefficient'])

Full Attributes CV Score: 0.2604353464190211


,coefficient
fixed acidity,0.005925
volatile acidity,-1.108038
citric acid,-0.263046
residual sugar,0.015322
chlorides,-1.730503
free sulfur dioxide,0.003801
total sulfur dioxide,-0.003899
density,4.338588
pH,-0.458535
sulphates,0.729719


    b)

In [8]:
att_score = []
for col in X.columns:
    att_score.append(np.mean(cross_val_score(SMWrapper(sm.OLS), X[col], y, scoring='r2', cv=5)))
df_score = pd.DataFrame(zip(X.columns, att_score),columns=['Attributes','r2 Score']).sort_values(by='r2 Score', ascending= False).reset_index(drop=True)
df_score.index = df_score.index+1
df_score

,Attributes,r2 Score
1,alcohol,0.163181
2,volatile acidity,0.028825
3,total sulfur dioxide,-0.073043
4,citric acid,-0.084692
5,chlorides,-0.129122
6,fixed acidity,-0.132844
7,pH,-0.139813
8,free sulfur dioxide,-0.143103
9,residual sugar,-0.147915
10,density,-0.162868


In [9]:
att_score = []
for col in X.columns:
    att_score.append(cross_val_score(SMWrapper(sm.OLS), X[col], y, scoring='r2', cv=4))
df_score_cv1 = pd.DataFrame(att_score,columns=['r2 Score fold1','r2 Score fold2','r2 Score fold3','r2 Score fold4'])
df_score_cv1['Attributes'] = X.columns
df_score_cv1 = df_score_cv1.sort_values(by='r2 Score fold1', ascending= False).reset_index(drop=True)
df_score_cv1.index = df_score_cv1.index+1
df_score_cv1

,r2 Score fold1,r2 Score fold2,r2 Score fold3,r2 Score fold4,Attributes
1,0.063166,0.197865,0.133434,0.226652,alcohol
2,-0.011582,-0.000635,0.048694,-0.014407,volatile acidity
3,-0.093504,0.001409,-0.143423,-0.223374,total sulfur dioxide
4,-0.144679,0.019359,-0.074247,-0.239200,citric acid
5,-0.155578,-0.007559,-0.164458,-0.358526,fixed acidity
6,-0.213676,-0.050337,-0.148017,-0.276457,chlorides
7,-0.215880,-0.045690,-0.139860,-0.310643,pH
8,-0.217667,-0.047050,-0.159717,-0.315076,free sulfur dioxide
9,-0.219697,-0.053777,-0.156278,-0.310569,residual sugar
10,-0.328743,-0.314689,-0.059804,-0.226508,density


In [10]:
df_score_cv2 = df_score_cv1.sort_values(by='r2 Score fold2', ascending= False).reset_index(drop=True)
df_score_cv2.index = df_score_cv2.index+1
df_score_cv2

,r2 Score fold1,r2 Score fold2,r2 Score fold3,r2 Score fold4,Attributes
1,0.063166,0.197865,0.133434,0.226652,alcohol
2,-0.520102,0.034172,-0.054130,-0.281518,sulphates
3,-0.144679,0.019359,-0.074247,-0.239200,citric acid
4,-0.093504,0.001409,-0.143423,-0.223374,total sulfur dioxide
5,-0.011582,-0.000635,0.048694,-0.014407,volatile acidity
6,-0.155578,-0.007559,-0.164458,-0.358526,fixed acidity
7,-0.215880,-0.045690,-0.139860,-0.310643,pH
8,-0.217667,-0.047050,-0.159717,-0.315076,free sulfur dioxide
9,-0.213676,-0.050337,-0.148017,-0.276457,chlorides
10,-0.219697,-0.053777,-0.156278,-0.310569,residual sugar


In [11]:
df_score_cv3 = df_score_cv1.sort_values(by='r2 Score fold3', ascending= False).reset_index(drop=True)
df_score_cv3.index = df_score_cv3.index+1
df_score_cv3

,r2 Score fold1,r2 Score fold2,r2 Score fold3,r2 Score fold4,Attributes
1,0.063166,0.197865,0.133434,0.226652,alcohol
2,-0.011582,-0.000635,0.048694,-0.014407,volatile acidity
3,-0.520102,0.034172,-0.054130,-0.281518,sulphates
4,-0.328743,-0.314689,-0.059804,-0.226508,density
5,-0.144679,0.019359,-0.074247,-0.239200,citric acid
6,-0.215880,-0.045690,-0.139860,-0.310643,pH
7,-0.093504,0.001409,-0.143423,-0.223374,total sulfur dioxide
8,-0.213676,-0.050337,-0.148017,-0.276457,chlorides
9,-0.219697,-0.053777,-0.156278,-0.310569,residual sugar
10,-0.217667,-0.047050,-0.159717,-0.315076,free sulfur dioxide


In [12]:
df_score_cv4 = df_score_cv1.sort_values(by='r2 Score fold4', ascending= False).reset_index(drop=True)
df_score_cv4.index = df_score_cv4.index+1
df_score_cv4

,r2 Score fold1,r2 Score fold2,r2 Score fold3,r2 Score fold4,Attributes
1,0.063166,0.197865,0.133434,0.226652,alcohol
2,-0.011582,-0.000635,0.048694,-0.014407,volatile acidity
3,-0.093504,0.001409,-0.143423,-0.223374,total sulfur dioxide
4,-0.328743,-0.314689,-0.059804,-0.226508,density
5,-0.144679,0.019359,-0.074247,-0.239200,citric acid
6,-0.213676,-0.050337,-0.148017,-0.276457,chlorides
7,-0.520102,0.034172,-0.054130,-0.281518,sulphates
8,-0.219697,-0.053777,-0.156278,-0.310569,residual sugar
9,-0.215880,-0.045690,-0.139860,-0.310643,pH
10,-0.217667,-0.047050,-0.159717,-0.315076,free sulfur dioxide


    c)

In [13]:
poly = PolynomialFeatures(degree= 2)
poly_x = poly.fit_transform(X)

In [14]:
cols = poly.get_feature_names()
cols[0] = 'bias'
for i in range(len(cols)):
    cols[i] = cols[i].replace(' ',' . ').replace('x10','alcohol')
    for j in range(len(X.columns)):
        cols[i] = cols[i].replace(f'x{j}',X.columns[j])

In [15]:
X2 = pd.DataFrame(poly_x,columns=cols)
X2.head()

,bias,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,...,density^2,density . pH,density . sulphates,density . alcohol,pH^2,pH . sulphates,pH . alcohol,sulphates^2,sulphates . alcohol,alcohol^2
0,1.0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,...,0.995605,3.502278,0.558768,9.37932,12.3201,1.9656,32.994,0.3136,5.264,88.36
1,1.0,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,...,0.993610,3.189760,0.677824,9.76864,10.2400,2.1760,31.360,0.4624,6.664,96.04
2,1.0,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,...,0.994009,3.250220,0.648050,9.77060,10.6276,2.1190,31.948,0.4225,6.370,96.04
3,1.0,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,...,0.996004,3.153680,0.578840,9.78040,9.9856,1.8328,30.968,0.3364,5.684,96.04
4,1.0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,...,0.995605,3.502278,0.558768,9.37932,12.3201,1.9656,32.994,0.3136,5.264,88.36


In [16]:
threshold = 0.05
max_pval = 1
drop_list =[]
model = None
model_score = None
while max_pval > threshold:
    X2_new = X2.drop(drop_list, axis=1)
    model = sm.OLS(y,X2_new).fit()
    model_score = np.mean(cross_val_score(SMWrapper(sm.OLS), X2_new, y, scoring='r2', cv=5))
    
    df_pval = pd.DataFrame(model.pvalues).reset_index()
    df_pval.columns=['Atrributes','Pvalues']
    df_pval = df_pval.sort_values(by='Pvalues')
    
    max_pval = np.max(df_pval['Pvalues'])
    max_pval_index = np.argmax(df_pval['Pvalues'])
    
    drop_att = df_pval.iloc[max_pval_index]['Atrributes']
    drop_list.append(drop_att)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                quality   R-squared (uncentered):                   0.989
Model:                            OLS   Adj. R-squared (uncentered):              0.988
Method:                 Least Squares   F-statistic:                              4122.
Date:                Mon, 31 Aug 2020   Prob (F-statistic):                        0.00
Time:                        17:44:23   Log-Likelihood:                         -1103.4
No. Observations:                1199   AIC:                                      2257.
Df Residuals:                    1174   BIC:                                      2384.
Df Model:                          25                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
fixed acidity                                -28.5463      6.443     -4.430      0.000     -41.188     -15.905
alcohol                                       26.3804      5.589      4.720      0.000      15.415      37.346
fixed acidity^2                               -0.0342      0.007     -4.792      0.000      -0.048      -0.020
fixed acidity . volatile acidity              -0.1319      0.060     -2.212      0.027      -0.249      -0.015
fixed acidity . chlorides                     -1.2871      0.368     -3.501      0.000      -2.008      -0.566
fixed acidity . density                       29.4212      6.562      4.483      0.000      16.546      42.296
volatile acidity . total sulfur dioxide        0.0107      0.003      3.698      0.000       0.005       0.016
volatile acidity . pH                         -0.9884      0.424     -2.330      0.020      -1.821      -0.156
volatile acidity . alcohol                     0.2762      0.136      2.034      0.042       0.010       0.543
citric acid . density                          6.6836      2.754      2.427      0.015       1.280      12.087
citric acid . pH                              -3.4931      0.832     -4.197      0.000      -5.126      -1.860
citric acid . alcohol                          0.4404      0.122      3.614      0.000       0.201       0.679
chlorides . density                            9.5536      3.124      3.058      0.002       3.424      15.683
free sulfur dioxide . total sulfur dioxide    -0.0002   6.97e-05     -2.510      0.012      -0.000   -3.82e-05
free sulfur dioxide . density                  0.0362      0.009      3.983      0.000       0.018       0.054
free sulfur dioxide . sulphates               -0.0272      0.011     -2.588      0.010      -0.048      -0.007
total sulfur dioxide^2                      4.318e-05   1.37e-05      3.148      0.002    1.63e-05    7.01e-05
total sulfur dioxide . density                -0.0126      0.003     -4.788      0.000      -0.018      -0.007
density^2                                    -18.4779      7.999     -2.310      0.021     -34.171      -2.784
density . pH                                   9.8320      4.615      2.131      0.033       0.778      18.886
density . alcohol                            -25.9870      5.560     -4.674      0.000     -36.896     -15.078
pH^2                                          -1.3282      0.675     -1.969      0.049      -2.652      -0.005
sulphates^2                                   -0.9392      0.221     -4.254      0.000      -1.372      -0.506
sulphates . alcohol                            0.2879      0.038      7.600      0.000       0.214

In [17]:
print(f'Final CV Score: {model_score}')
df_final = pd.DataFrame(model.params, columns=['coefficient'])
df_final

Final CV Score: 0.325033020959744


,coefficient
fixed acidity,-28.546312
alcohol,26.380443
fixed acidity^2,-0.034238
fixed acidity . volatile acidity,-0.131926
fixed acidity . chlorides,-1.287134
fixed acidity . density,29.421214
volatile acidity . total sulfur dioxide,0.010700
volatile acidity . pH,-0.988443
volatile acidity . alcohol,0.276158
citric acid . density,6.683647
